In [106]:
import numpy as np

#Définition d'un line world sous la forme d'un MDP

In [107]:
S = [0, 1, 2 ,3, 4]
A = [0, 1] # left right
R = [-1, 0, 1]
T = [0, 4]
p = np.zeros((len(S), len(A), len(S), len(R))) # S, A, S_p, R

In [108]:
for s in range(len(S)):
  for a in range(len(A)):
    for s_p in range(len(S)):
      for r in range(len(R)):
        if s_p == s + 1 and a == 1 and R[r] == 0 and s in [1, 2]:
          p[s, a, s_p, r] = 1.0
        if s_p == s - 1 and a == 0 and R[r] == 0 and s in [2, 3]:
          p[s, a, s_p, r] = 1.0
p[3, 1, 4, 2] = 1.0
p[1, 0, 0, 0] = 1.0

print(p)

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[1. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 1. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 1. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 1. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 1. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 1.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


# Exercice

Implémentez une fonction prenant en paramètre une policy (un tableau numpy)
et renvoyant la value de cette policy (un tableau numpy) pour chaque état du line world

In [109]:
def policy_evaluation(policy : np.ndarray, V: np.ndarray = None) -> np.ndarray:
  theta = 0.00001
  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0
  gamma = 0.999
  while True:
    delta = 0
    for s in range(len(S)):
      v = V[s]
      total = 0
      for a in range(len(A)):
        pi_s_a = policy[s, a]
        for s_p in range(len(S)):
          for r in range(len(R)):
            total += pi_s_a * p[s, a, s_p, r] * (R[r] + 0.999 * V[s_p])
      V[s] = total
      delta = np.maximum(delta, np.abs(v - V[s]))
    if delta < theta:
      return V

Définissez une policy jouant tout le temps à droite.
Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [110]:
pi_right = np.array([[0.0, 1.0] for _ in range(len(S))])
print(policy_evaluation(pi_right))

[0.       0.998001 0.999    1.       0.      ]


Définissez une policy jouant tout le temps à gauche. Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [111]:
pi_left = np.array([[1.0, 0.0] for _ in range(len(S))])
print(policy_evaluation(pi_left))

[ 0.       -1.       -0.999    -0.998001  0.      ]


Définissez une policy uniformément aléatoire (50% de chance d'aller à gauche et 50% de chances d'aller à droite). Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [112]:
pi_random = np.array([[0.5, 0.5] for _ in range(len(S))])
print(policy_evaluation(pi_random))

[ 0.00000000e+00 -4.99992999e-01  6.99360232e-06  5.00003493e-01
  0.00000000e+00]


Définissez une policy uniformément aléatoire (15% de chance d'aller à gauche et 85% de chances d'aller à droite). Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [113]:
pi_weird_random = np.array([[0.15, 0.85] for _ in range(len(S))])
print(policy_evaluation(pi_weird_random))

[0.         0.64651341 0.93801507 0.99056156 0.        ]


In [114]:
def naive_ineficient_policy_iteration():
  theta = 0.000001
  gamma = 0.999

  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0

  Pi = np.random.choice(A, len(S))

  while True:
    # Policy Evaluation
    while True:
      delta = 0.0

      for s in S:
        v = V[s]

        total = 0.0
        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, Pi[s], s_p, r_index] * (R[r_index] + gamma * V[s_p])

        V[s] = total

        delta = max(delta, abs(v - V[s]))

      if delta < theta:
        break

    # Policy Improvement
    policy_stable = True

    for s in S:
      old_action = Pi[s]

      best_a = None
      best_action_score = -9999999

      for a in A:
        total = 0.0
        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, a, s_p, r_index] * (R[r_index] + gamma * V[s_p])

        if best_a is None or total >= best_action_score:
          best_a = a
          best_action_score = total

      Pi[s] = best_a
      if Pi[s] != old_action:
        policy_stable = False

    if policy_stable:
      return Pi

In [115]:
naive_ineficient_policy_iteration()

array([1, 1, 1, 1, 1])

# Définissez une variante d'un GRID WORLD sous la forme d'un MDP et évaluez différentes stratégies sur cette environnement.

Le grid world est une grille de 5x5 cases (5 lignes de 5 colonnes) sur laquelle l'agent peut évoluer, il commence généralement sur la première ligne, première colonne. L'agent possède 4 actions possibles (gauche, droite, haut, bas). Si jamais l'agent atteint la dernière case de la première ligne => état terminal avec reward de -3. Si jamais l'agent atteint la dernière case de la dernière ligne => état terminal avec reward de 1. Si l'agent essaye de se déplacer en dehors des bords de la grille => état terminal avec score de -1.

In [116]:
import numpy as np

# Define the grid world size
S = np.zeros((5, 5))

# Define the possible actions, order: [up, left, down, right]
A = np.array([[-1, 0], [0, -1], [1, 0], [0, 1]])

# Define the rewards for terminal states
R = {(0, 4): -3, (4, 4): 1}

# Define the transition function
def transition(state, action):
    row, col = state
    new_row, new_col = row + action[0], col + action[1]

    # Check for out of bounds - returning to the same state with a reward of -1
    if new_row < 0 or new_row >= len(S) or new_col < 0 or new_col >= len(S):
        return state, -1, True

    # Check for the specific terminal states with custom rewards
    if (new_row, new_col) in R:
        return (new_row, new_col), R[(new_row, new_col)], True

    # If the new state is not terminal, return the new state, a reward of 0, and False for non-terminal
    return (new_row, new_col), 0, False

# Initialize the transition probability matrix, correct dimensions this time
p = np.zeros((len(S), len(S[0]), len(A), len(S), len(S[0])))

# Fill in the transition probability matrix
for row in range(len(S)):
    for col in range(len(S[0])):
        for action_idx, action in enumerate(A):
            next_state, reward, done = transition((row, col), action)
            if done:
                # Assign a probability of 1 to remain in or move to the terminal state
                p[row, col, action_idx, next_state[0], next_state[1]] = 1
            else:
                # Assign a probability of 1 to move to the next valid state
                p[row, col, action_idx, next_state[0], next_state[1]] = 1



# Proposez plusieurs stratégies et évaluez les à l'aide de policy évaluation.

In [118]:
import numpy as np

# Define the grid world size
S = np.zeros((5, 5))

# Define possible actions, order: [up, left, down, right]
A = np.array([[-1, 0], [0, -1], [1, 0], [0, 1]])

# Define the rewards for terminal states
R = {(0, 4): -3, (4, 4): 1}

def index_to_pos(index):
    """ Convert flat index to grid position """
    return (index // 5, index % 5)

def pos_to_index(row, col):
    """ Convert grid position to flat index """
    return row * 5 + col

# Example transition probabilities (simplified)
p = np.zeros((25, 4, 25))  # s, a, s'

# Simplified transition model assuming deterministic outcomes
for s in range(25):
    pos = index_to_pos(s)
    for a in range(4):
        new_pos = (pos[0] + A[a, 0], pos[1] + A[a, 1])
        if 0 <= new_pos[0] < 5 and 0 <= new_pos[1] < 5:
            s_p = pos_to_index(*new_pos)
        else:
            s_p = s  # remains in the same state if out of bounds
        p[s, a, s_p] = 1

def policy_evaluation(policy, V=None):
    theta = 0.00001
    gamma = 0.999

    if V is None:
        V = np.zeros(25)  # Match number of states

    while True:
        delta = 0
        for s in range(25):
            v = V[s]
            total = 0
            for a in range(4):
                pi_s_a = policy[s, a]
                for s_p in range(25):
                    pos = index_to_pos(s_p)
                    reward = R.get(pos, 0)
                    total += pi_s_a * p[s, a, s_p] * (reward + gamma * V[s_p])
            V[s] = total
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            return V

# Define policies
pi_right = np.zeros((25, 4))
pi_right[:, 3] = 1  # Assume right is the 3rd action

pi_left = np.zeros((25, 4))
pi_left[:, 1] = 1  # Assume left is the 1st action

pi_up = np.zeros((25, 4))
pi_up[:, 0] = 1  # Assume up is the 0th action

pi_down = np.zeros((25, 4))
pi_down[:, 2] = 1  # Assume down is the 2nd action

# Evaluate policies
print("Policy Evaluation for Right:")
print(policy_evaluation(pi_right))
print("Policy Evaluation for Left:")
print(policy_evaluation(pi_left))
print("Policy Evaluation for Up:")
print(policy_evaluation(pi_up))
print("Policy Evaluation for Down:")
print(policy_evaluation(pi_down))


Policy Evaluation for Right:
[-2990.99901468 -2993.99301768 -2996.99001768 -2999.99001768
 -2999.99001768     0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
   996.99967156   997.99767256   998.99667256   999.99667256
   999.99667256]
Policy Evaluation for Left:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
Policy Evaluation for Up:
[    0.             0.             0.             0.
 -2999.99001768     0.             0.             0.
     0.         -2999.99002767     0.             0.
     0.             0.         -2996.99003764     0.
     0.             0.             0.         -2993.9930476
     0.             0.             0.             0.
 -2990.99905455]
Policy Evaluation for Down:
[  0.           0.           0.           0.         996.99301731
   0.           0.           0.           0.        

# Bonus : Implémentez policy itération et value itération (dans les slides que nous n'avons pas encore vu) et obtenez à l'aide de ces derniers pi* pour le Line World et le Grid World

In [ ]:
#TODO

# Naive Policy iteration implementation in order to find a Pi*

In [ ]:
def naive_policy_iteration():
  theta = 0.00001
  gamma = 0.999

  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0
  Pi = np.random.choice(A, len(S), True)

  while True:
    # Policy evaluation
    while True:
      delta = 0
      for s in range(len(S)):
        v = V[s]
        total = 0
        for s_p in range(len(S)):
          for r in range(len(R)):
            total += p[s, Pi[s], s_p, r] * (R[r] + 0.999 * V[s_p])
        V[s] = total
        delta = np.maximum(delta, np.abs(v - V[s]))
      if delta < theta:
        break

    # Policy improvement
    policy_stable = True

    for s in S:
      if s in T:
        continue

      old_action = Pi[s]

      # Compute Arg Max a
      argmax_a = None
      max_a = -999999999

      for a in A:
        total = 0.0

        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, a, s_p, r_index] * (R[r_index] + gamma * V[s_p])

        if argmax_a is None or total >= max_a:
          argmax_a = a
          max_a = total

      Pi[s] = argmax_a

      if old_action != Pi[s]:
        policy_stable = False


    if policy_stable:
      break

  return Pi


In [ ]:
naive_policy_iteration()

array([0, 0, 0, 0, 1])